In [1]:
# Import the fucking useful libraries =)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.sparse as sp
import itertools
import math
from MF_SGD import *



# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [37]:
from helpers_scipy import load_data, percent_sparse
data = load_data('../data/data_train.csv')
print("percent of defined matrix", percent_sparse(data))
print("# non-zeros", data.nnz)

number of users: 10000, number of movies: 1000
percent of defined matrix 8.496523222697272
# non-zeros 1176952


In [23]:
data.shape

(10000, 1000)

In [18]:
row, col, rat = sp.find(data)

In [22]:
ro

array([   44,    61,    67, ...,  9994,  9997, 10000], dtype=int32)

In [20]:
col + 1

array([   1,    1,    1, ..., 1000, 1000, 1000], dtype=int32)

In [42]:
def sp_to_df(sparse):
    row, col, rat = sp.find(sparse)
    row += 1
    col += 1

    df = pd.DataFrame({'User':row,'Movie':col,'Rating':rat})
    df = df[['User', 'Movie', 'Rating']].sort_values(['Movie', 'User'])
    return df
    
df = sp_to_df(data)
df.head()

,User,Movie,Rating
0,44,1,4.0
1,61,1,3.0
2,67,1,4.0
3,72,1,3.0
4,86,1,5.0


In [64]:
user = df['User']
movie = df['Movie']
rating = df['Rating']

for u, m, r in zip (user, movie, rating):
    mat[u-1, m-1] = r 

In [113]:
df

,User,Movie,Rating
0,44,1,4.0
1,61,1,3.0
2,67,1,4.0
3,72,1,3.0
4,86,1,5.0
5,90,1,4.0
6,108,1,3.0
7,114,1,3.0
8,120,1,2.0
9,135,1,5.0


In [127]:

user_mean = df.groupby('User').mean()['Rating']

def line(df):
    df['Rating'] = user_mean[df['User']]
    return df[['User', 'Movie', 'Rating']]

df[:50].apply(line, axis=1)

,User,Movie,Rating
0,44.0,1.0,3.893701
1,61.0,1.0,4.233696
2,67.0,1.0,3.580645
3,72.0,1.0,3.861423
4,86.0,1.0,4.103321
5,90.0,1.0,3.877698
6,108.0,1.0,3.775229
7,114.0,1.0,4.190000
8,120.0,1.0,2.958974
9,135.0,1.0,4.198880


In [119]:
user_mean = df.groupby('User').mean()
user_mean

,Movie,Rating
User,,
1,512.913043,4.043478
2,497.765101,3.771812
3,420.930233,3.523256
4,383.300000,3.800000
5,478.679104,3.985075
6,468.881657,4.366864
7,465.919355,4.129032
8,301.884615,4.115385
9,494.837079,3.971910


In [106]:
df['Rating']

0          4.0
1          3.0
2          4.0
3          3.0
4          5.0
5          4.0
6          3.0
7          3.0
8          2.0
9          5.0
10         4.0
11         3.0
12         3.0
13         3.0
14         1.0
15         3.0
16         2.0
17         4.0
18         4.0
19         2.0
20         3.0
21         2.0
22         4.0
23         3.0
24         3.0
25         3.0
26         2.0
27         3.0
28         4.0
29         1.0
          ... 
1176922    3.0
1176923    4.0
1176924    3.0
1176925    5.0
1176926    3.0
1176927    2.0
1176928    5.0
1176929    5.0
1176930    4.0
1176931    2.0
1176932    3.0
1176933    3.0
1176934    4.0
1176935    2.0
1176936    4.0
1176937    5.0
1176938    4.0
1176939    3.0
1176940    4.0
1176941    3.0
1176942    4.0
1176943    3.0
1176944    5.0
1176945    5.0
1176946    5.0
1176947    4.0
1176948    5.0
1176949    3.0
1176950    4.0
1176951    3.0
Name: Rating, dtype: float64

In [138]:
from means import *
pred = user_mean(df,df[:50])
pred.head()

,User,Movie,Rating
0,44,1,3.893701
1,61,1,4.233696
2,67,1,3.580645
3,72,1,3.861423
4,86,1,4.103321


In [146]:
df['Rating'].mean()

3.8572805008190647

In [147]:
from means import *
pred = global_mean(df,df[:50])
pred.head()

,User,Movie,Rating
0,44,1,3.857281
1,61,1,3.857281
2,67,1,3.857281
3,72,1,3.857281
4,86,1,3.857281
